In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime


from datetime import timedelta, datetime, date
import calendar

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import glob


# This is my preprocessing function, which I run on each grib file

# Load, preprocess, append, and concatenate all the grib files. In this notebook I ran it on half of the data available.

### I will comment the code below for pr_sfc. I repeat the code for each variable separiately. I could loop it for a list of varname =[ 'pr_sfc' , ....] but I preferred keeping it separate.

In [2]:
# from dask.distributed import Client
# # just have dask figure everything out
# client = Client()
# client

In [3]:
from glob import glob
variable_names = ['cape', 'tas_2m', 'tdps_sfc', 'wap_500']

def ppf2_filename(ds,filename):
    # rename coordinates
    ds  = ds.rename({ 'lon_0':'lon', 'lat_0':'lat', 'forecast_time0':'time'})
    # getting start time from file name
    ts = filename.split('_')
#     print(ts)
    whereGEFS = ts.index('GEFS')
#     print(whereGEFS)
    dateSstr = ts[whereGEFS+1]
#     print(dateSstr)
    dateS = datetime.strptime(dateSstr, '%d%b%Y')
#     print(dateS)
    
    Mvalue = np.int(float(np.asarray(ts[-1][2:4])))

    ds.coords['S'] = 'S', np.atleast_1d(dateS)
    ds.coords['M'] = 'M', np.atleast_1d(Mvalue)
    
    ds.coords['time']=np.array([  43200000000000,  129600000000000,  216000000000000,
        302400000000000,  388800000000000,  475200000000000,
        561600000000000,  648000000000000,  734400000000000,
        820800000000000,  907200000000000,  993600000000000,
       1080000000000000, 1166400000000000, 1252800000000000,
       1339200000000000, 1425600000000000, 1512000000000000,
       1598400000000000, 1684800000000000, 1771200000000000,
       1857600000000000, 1944000000000000, 2030400000000000,
       2116800000000000, 2203200000000000, 2289600000000000,
       2376000000000000, 2462400000000000, 2548800000000000,
       2635200000000000, 2721600000000000, 2808000000000000,
       2894400000000000, 2980800000000000],
      dtype='timedelta64[ns]')

    # select US
    ds = ds.sel(lat=slice(50,23), lon=slice(230,300))
#     print(ds)
    return ds

def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
#         print(path)
        with xr.open_dataset(path, engine='pynio', chunks={}) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds
#     print(files)
    paths = sorted(glob(files))
#     print(paths)
    datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined

var_l=[]
for ivx, ivar in enumerate(variable_names):
    print(ivar)
    if ivar in os.listdir('/Data2/SubX/EMC/GEFS/'):
        c_l = []
        for iy in np.arange(1999, 2017):#2017
            print(iy)
            # here we suppose we only care about the combined mean of each file;
            # you might also use indexing operations like .sel to subset datasets
            combined = read_netcdfs('/Data2/SubX/EMC/GEFS/'+ivar+'/'+np.str(iy)+'/*/*.grb2', dim=['S','M'],
                                    transform_func=ppf2_filename)
            
            c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)


cape
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
tas_2m
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
tdps_sfc
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
wap_500
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [16]:
all_variables = xr.merge(var_l)
all_variables

<xarray.Dataset>
Dimensions:             (M: 10, S: 939, lat: 28, lon: 71, time: 35)
Coordinates:
  * lat                 (lat) float32 50.0 49.0 48.0 47.0 ... 25.0 24.0 23.0
  * lon                 (lon) float32 230.0 231.0 232.0 ... 298.0 299.0 300.0
  * time                (time) timedelta64[ns] 0 days 12:00:00 ... 34 days 12:00:00
  * M                   (M) int64 0 1 2 3 4 5 6 7 8 9
  * S                   (S) datetime64[ns] 1999-01-06 1999-01-13 ... 2016-12-28
Data variables:
    CAPE_P1_2L108_GLL0  (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    TMP_P1_L103_GLL0    (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    DPT_P1_L103_GLL0    (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    VVEL_P1_L100_GLL0   (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>

In [17]:
all_variables = all_variables.rename({'CAPE_P1_2L108_GLL0':'cape',
                                      'TMP_P1_L103_GLL0':'tas_2m',
                                      'DPT_P1_L103_GLL0':'tdps_2m',
                                      'VVEL_P1_L100_GLL0':'wap_500'})

In [18]:
all_variables

<xarray.Dataset>
Dimensions:  (M: 10, S: 939, lat: 28, lon: 71, time: 35)
Coordinates:
  * lat      (lat) float32 50.0 49.0 48.0 47.0 46.0 ... 27.0 26.0 25.0 24.0 23.0
  * lon      (lon) float32 230.0 231.0 232.0 233.0 ... 297.0 298.0 299.0 300.0
  * time     (time) timedelta64[ns] 0 days 12:00:00 ... 34 days 12:00:00
  * M        (M) int64 0 1 2 3 4 5 6 7 8 9
  * S        (S) datetime64[ns] 1999-01-06 1999-01-13 ... 2016-12-21 2016-12-28
Data variables:
    cape     (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    tas_2m   (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    tdps_2m  (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    wap_500  (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>

In [14]:
all_variables

<xarray.Dataset>
Dimensions:  (M: 10, S: 939, lat: 28, lon: 71, time: 35)
Coordinates:
  * lat      (lat) float32 50.0 49.0 48.0 47.0 46.0 ... 27.0 26.0 25.0 24.0 23.0
  * lon      (lon) float32 230.0 231.0 232.0 233.0 ... 297.0 298.0 299.0 300.0
  * time     (time) timedelta64[ns] 0 days 12:00:00 ... 34 days 12:00:00
  * M        (M) int64 0 1 2 3 4 5 6 7 8 9
  * S        (S) datetime64[ns] 1999-01-06 1999-01-13 ... 2016-12-21 2016-12-28
Data variables:
    cape     (S, M, time, lat, lon) float32 2.0 4.0 7.0 2.0 ... 39.0 32.0 25.0
    tas_2m   (S, M, time, lat, lon) float32 281.35 281.5 ... 296.66998 296.6
    tdps_2m  (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>
    wap_500  (S, M, time, lat, lon) float32 dask.array<chunksize=(4, 10, 35, 28, 71), meta=np.ndarray>

In [20]:
# for iv in ['cape', 'tas_2m', 'tdps_2m', 'wap_500']:
# for iv in ['tas_2m', 'tdps_2m', 'wap_500']:
for iv in ['cape', 'tas_2m','tdps_2m', 'wap_500']:
    var_temp = all_variables[[iv]].load()
    all_variables_3L = var_temp.rolling(time=3).sum()
    all_variables_3L.coords['time'] = all_variables_3L.time.values-np.timedelta64(2,'D')
    all_variables_3L = all_variables_3L.isel(time=slice(2,36))
    all_variables_3L.coords['target_time'] = all_variables_3L.S+all_variables_3L.time
    all_variables_3L.to_netcdf('all_variables_3L_oct2020_'+iv+'.nc')



    